In [0]:
"""
03_silver_cleaning.py

Purpose:
- Parse raw JSON from Bronze layer
- Enforce schema and data types
- Clean and deduplicate sensor measurements
- Write structured Silver Delta table

Input:
- air_quality_bronze.live_sensor_raw

Output:
- air_quality_silver.sensor_measurements

Visual tree view of the sensor_schema:
sensor_schema
├─ id : Long
├─ timestamp : String
├─ location : Struct
│    ├─ id : Long
│    ├─ latitude : String
│    ├─ longitude : String
│    └─ country : String
├─ sensor : Struct
│    ├─ id : Long
│    └─ sensor_type : Struct
│          └─ name : String
└─ sensordatavalues : Array of Struct
     ├─ value_type : String
     └─ value : String

"""

# Databricks notebooks do not automatically include the project root in PYTHONPATH.
# To enable imports from src/, each notebook adds the project root to sys.path at runtime
import sys, os
sys.path.append(os.path.abspath(".."))

# Ensure silver database exists
spark.sql("CREATE DATABASE IF NOT EXISTS air_quality_silver")

# Read bronze table
from pyspark.sql.functions import (
    col, 
    from_json, 
    to_timestamp, 
    explode, 
    first, 
    when,
    row_number)
from pyspark.sql.types import (
    StructType, 
    StructField, 
    StringType, 
    DoubleType, 
    LongType, 
    ArrayType)
from pyspark.sql.window import Window

bronze_df = spark.read.table("air_quality_bronze.live_sensor_raw")

# Define JSON schema
sensor_schema = StructType([
    StructField("id", LongType(), True),

    StructField("timestamp", StringType(), True),
    
    StructField("location", StructType([
        StructField("id", LongType(), True),
        StructField("latitude", StringType(), True),
        StructField("longitude", StringType(), True),
        StructField("country", StringType(), True)
    ]), True),

    StructField("sensor", StructType([
        StructField("id", LongType(), True),
        StructField("sensor_type", StructType([
            StructField("name", StringType(), True)
        ]), True)
    ]), True),

    StructField(
        "sensordatavalues",
        ArrayType(
            StructType([
                StructField("value_type", StringType(), True),
                StructField("value", StringType(), True)
            ])
        ),
        True
    )
])

# Parse JSON & explode onto structured columns
parsed_df = bronze_df.withColumn(
    "parsed_json",
    from_json(col("raw_json"), sensor_schema)
)

# Select & clean silver columns
exploded_df = parsed_df.select(
    col("parsed_json.sensor.id").alias("sensor_id"),
    col("parsed_json.sensor.sensor_type.name").alias("sensor_type"),
    to_timestamp(col("parsed_json.timestamp")).alias("measurement_ts"),
    col("parsed_json.location.id").alias("location_id"),
    col("parsed_json.location.latitude").cast("double").alias("latitude"),
    col("parsed_json.location.longitude").cast("double").alias("longitude"),
    col("parsed_json.location.country").alias("country"),
    explode(col("parsed_json.sensordatavalues")).alias("measurement"),
    col("ingested_at"),
    col("batch_id")
)

# Silver projection
silver_df = exploded_df.select(
    "sensor_id",
    "sensor_type",
    "measurement_ts",
    "location_id",
    "latitude",
    "longitude",
    "country",
    col("measurement.value_type").alias("measurement_type"),
    col("measurement.value").cast("double").alias("measurement_value"),
    "ingested_at",
    "batch_id"
).filter(
    col("measurement_ts").isNotNull() &
    col("measurement_value").isNotNull()
)

# Filter invalid records (light cleaning)
silver_df_clean = silver_df.filter(
    col("sensor_id").isNotNull() &
    col("measurement_ts").isNotNull()
)

# Deduplicate deterministically - One row per (sensor_id, timestamp, measurement_type) and keep latest known version
dedup_window = Window.partitionBy(
    "sensor_id",
    "measurement_ts",
    "measurement_type"
).orderBy(
    col("ingested_at").desc(), # keep latest ingested record
    col("batch_id").desc()
)

silver_df_dedup = (
    silver_df_clean
    .withColumn("row_num", row_number().over(dedup_window))
    .filter(col("row_num") == 1)
    .drop("row_num")
)

# Add quality checks / plausibility filtering
# Create separate PM10 / PM2.5 columns to check ratio
pm_df = silver_df_dedup.filter(col("measurement_type").isin(["P1", "P2"]))

pm_pivot_df = pm_df.groupBy(
    "location_id",
    "measurement_ts",
).pivot("measurement_type", ["P1", "P2"]).agg(first("measurement_value"))

# Add plausibility flag for ratio: PM2.5 <= 1.2 * PM10
pm_pivot_df = pm_pivot_df.withColumn(
    "ratio_plausible",
    when(
        col("P1").isNotNull() & col("P2").isNotNull(),
        col("P2") <= col("P1") * 1.2
    ).otherwise(True)
)

# Set explicit PM10 / PM2.5 values as upper bonds
MAX_PM10 = 1000.0
MAX_PM25 = 500.0

# Merge back ratio flag with main silver_df and add quality flag case expressions
silver_df_final = silver_df_dedup.join(
    pm_pivot_df.select(
        "location_id",
        "measurement_ts",
        "ratio_plausible"
    ),
    on=["location_id", "measurement_ts"],
    how="left"
).withColumn(
    "quality_reason",
    when(col("measurement_value") < 0, "NEGATIVE_VALUE")
    .when(
        (col("measurement_type") == "P1") & (col("measurement_value") > MAX_PM10),
        "PM10_OUT_OF_RANGE"
    )
    .when(
        (col("measurement_type") == "P2") & (col("measurement_value") > MAX_PM25),
        "PM25_OUT_OF_RANGE"
    )
    .when(
        col("measurement_type").isin("P1", "P2") &
        (~col("ratio_plausible")),
        "PM_RATIO_VIOLATION"
    )
    .otherwise("OK")
).withColumn(
    "quality_flag",
    when(col("quality_reason") == "OK", "OK").otherwise("BAD")
)

# Write Silver Delta table
SILVER_TABLE = "air_quality_silver.sensor_measurements"

silver_df_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(SILVER_TABLE)

# Silver layer schema check
display(
    spark.sql(
        f"""
        DESCRIBE {SILVER_TABLE}"""
    )
)

# Silver layer cardinality check
display(
    spark.sql(
        f"""
        SELECT
            COUNT(*) AS rows,
            COUNT(DISTINCT sensor_id) AS sensors,
            COUNT(DISTINCT location_id) AS locations
        FROM {SILVER_TABLE}"""
    )
)

# Silver layer sanity check 1: Verify which sensors exist & whether PM-data exists
display(
    spark.sql(
        f"""
        SELECT
            measurement_type,
            COUNT(*) AS records
        FROM {SILVER_TABLE}
        GROUP BY measurement_type
        ORDER BY records DESC
        """            
    )
)

# Silver layer sanity check 2: Check PM-only aggregation
display(
    spark.sql(
        f"""
        SELECT
            sensor_type,
            measurement_type,
            COUNT(*) AS records,
            AVG(measurement_value) AS avg_value
        FROM {SILVER_TABLE}
        WHERE measurement_type IN ('P1', 'P2')
        GROUP BY sensor_type, measurement_type
        ORDER BY records DESC
        """
    )
)

# Silver layer sanity check 3: Check ratio plausibility
display(
    spark.sql(
        f"""
        SELECT
            sensor_type,
            measurement_type,
            ratio_plausible,
            quality_reason,
            quality_flag
        FROM {SILVER_TABLE}
        WHERE measurement_type IN ('P1', 'P2')
        """        
    )
)

silver_rows = spark.read.table(
    "air_quality_silver.sensor_measurements"
).count()

dbutils.notebook.exit(
    f"Silver cleaning completed: {silver_rows} records in silver table"
)


col_name,data_type,comment
location_id,bigint,null
measurement_ts,timestamp,null
sensor_id,bigint,null
sensor_type,string,null
latitude,double,null
longitude,double,null
country,string,null
measurement_type,string,null
measurement_value,double,null
ingested_at,timestamp,null


rows,sensors,locations
1494971,21792,12522


measurement_type,records
P2,296740
P1,296727
temperature,245410
humidity,234633
pressure,142138
pressure_at_sealevel,140620
P0,21751
P4,13924
N1,13869
N05,13752


sensor_type,measurement_type,records,avg_value
SDS011,P2,272765,23.587622202261592
SDS011,P1,272752,45.01314916114455
SPS30,P1,12037,23.930204369859574
SPS30,P2,12037,22.97838414887429
PMS7003,P1,5315,47.83149012229539
PMS7003,P2,5315,40.26974600188153
PMS5003,P1,5243,22.704939919893178
PMS5003,P2,5243,19.883284379172252
SEN5x,P2,555,31.05034234234234
SEN5x,P1,555,32.91142342342339


sensor_type,measurement_type,ratio_plausible,quality_reason,quality_flag
PPD42NS,P1,true,PM10_OUT_OF_RANGE,BAD
PPD42NS,P2,true,OK,OK
PPD42NS,P1,true,OK,OK
PPD42NS,P2,true,OK,OK
PPD42NS,P1,true,OK,OK
PPD42NS,P2,true,OK,OK
PPD42NS,P2,true,OK,OK
PPD42NS,P1,true,OK,OK
PPD42NS,P2,true,OK,OK
PPD42NS,P1,true,OK,OK
